# Importing Libraries 

In [ ]:
import pandas as pd
import numpy as np
import math
# import time 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import gmplot
from __future__ import division
from __future__ import print_function
import requests
import json
import urllib
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from requests.structures import CaseInsensitiveDict
from IPython.display import display, HTML

import random

# Importing Data 

In [ ]:
loca = pd.read_excel('Final_locality.xlsx')
ad = pd.read_excel('updated_ad.xlsx')

In [ ]:

Bhopal_localities = loca.loc[(loca.City == 'Bhopal')& (loca.wrong_data != True)]
Bhopal_localities.drop(columns = 'Unnamed: 0',inplace = True)


# Initializing Random Sample Data of 40 rows

In [ ]:
Bhopal_ad = ad.loc[(ad.LocalityId.isin(Bhopal_localities.Id).astype(int)) &(ad.difference >=0) &(ad.Error == False) ]
Sample_ad=Bhopal_ad.sample(n=40)

# Adding Pincode Column
for index, row in Sample_ad.iterrows():
    pin = loca[loca.Id == row['LocalityId']].pincode.values[0]
    Sample_ad.at[index,'Pincode'] = pin

# Variable with the Longitude and Latitude
Sample =Sample_ad.loc[:,['Id','Latitude','Longitude','Pincode']]
#Fixing Index of Sample
Sample.reset_index(drop=True, inplace=True)


## Creating Distance Matrix for with Haversine Distance

In [5]:
def create_data_model(vehicle_no=4):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = []
    dist = []
    for i in range(len(Sample)):
        matrix_list=[]
        for j in range(len(Sample)):
            d1 = Sample.iloc[i]
            d2 = Sample.iloc[j]
            lat1 = d1.Latitude
            lon1 = d1.Longitude
            lat2 = d2.Latitude
            lon2 = d2.Longitude
            dist.append(haversine_distance(lat1, lon1, lat2, lon2))
        data['distance_matrix'].append(dist)
        dist=[]
    data['num_vehicles'] = vehicle_no
    data['depot'] = 0
    return data

## Open source Distance Matrix OSRM 

#### https://github.com/Project-OSRM/osrm-backend/blob/master/docs/http.md

In [14]:
def open_dm(vehicle_no=4,data=None):
    Sample_x = data[:]
    str_x =''
    for x, y in zip(Sample_x['Latitude'], Sample_x['Longitude']):
        str_x+=f'{y},{x};'
    str_x = str_x[:-1]
    
    # url_x gives distace matrix 
    url_x = f"http://router.project-osrm.org/table/v1/driving/{str_x}?annotations=distance"
    resp = requests.post(url_x)
    data_x = json.loads(resp.text)
    data = {}
    lol = data_x['distances']
    for i in range(len(lol)):
        for j in range(len(lol[i])):
            lol[i][j]=lol[i][j]/1000
    data['distance_matrix'] = lol[:]
    data['num_vehicles'] = vehicle_no
    data['depot'] = 0
    data['Id'] = Sample.to_dict()['Id']
    return data

# Vehicle Routing Problem Implementation

In [15]:
def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them xin an array."""
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

In [16]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}Km\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}Km'.format(max_route_distance))

In [17]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

# Original VRP w/o any constrainsts  

In [31]:
vehicle_no = 4 
df=Sample[:]
data = open_dm(vehicle_no,df)
data['starts'] =[0,0,0,0]
data['ends'] = [0,0,0,0]
print((len(data['distance_matrix']),data['num_vehicles'], data['starts'],data["ends"]))
#print(data)
# Create the routing index manager.
# if len(start)!=vehicle_no or len(end)!= vehicle_no:
#     print('Problem with Start and End \n Initializing start and end points of zero')
#     manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),data['num_vehicles'], data['depot'])
# else:
#     manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),data['num_vehicles'], data['starts'],data["ends"])

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),data['num_vehicles'], data['starts'],data["ends"])
routing = pywrapcp.RoutingModel(manager)

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    10000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)
# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
else:
    print('No Soluchan for you')
get_routes(solution, routing, manager)

(40, 4, [0, 0, 0, 0], [0, 0, 0, 0])
Route for vehicle 0:
 0 ->  24 ->  27 ->  6 ->  20 ->  34 ->  18 ->  5 ->  3 ->  35 ->  17 ->  26 ->  37 ->  38 -> 0
Distance of the route: 35Km

Route for vehicle 1:
 0 ->  4 ->  14 ->  25 ->  22 ->  28 ->  21 -> 0
Distance of the route: 38Km

Route for vehicle 2:
 0 ->  8 ->  23 ->  9 ->  39 ->  13 ->  16 ->  2 ->  7 -> 0
Distance of the route: 40Km

Route for vehicle 3:
 0 ->  12 ->  19 ->  31 ->  29 ->  11 ->  15 ->  10 ->  36 ->  1 ->  30 ->  32 ->  33 -> 0
Distance of the route: 39Km

Maximum of the route distances: 40Km


[[0, 24, 27, 6, 20, 34, 18, 5, 3, 35, 17, 26, 37, 38, 0],
 [0, 4, 14, 25, 22, 28, 21, 0],
 [0, 8, 23, 9, 39, 13, 16, 2, 7, 0],
 [0, 12, 19, 31, 29, 11, 15, 10, 36, 1, 30, 32, 33, 0]]

In [33]:
def show_solution(vehicle_no,df,start=[],end = []):
    if len(start)!=vehicle_no or len(end)!= vehicle_no:
        print('length of start list/ endlist does not match with number of vehicle')
        print('initializing start and end to 0')
        start = [0 for x in range(vehicle_no)]
        
        end = [0 for x in range(vehicle_no)]
    
    data = open_dm(vehicle_no,df)
    data['starts'] =start[:]
    data['ends'] = end[:]
    print('Start_list:',data['starts'])
    print('Ends_list:',data['ends'])
    
    
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),data['num_vehicles'], data['starts'],data["ends"])
    
    routing = pywrapcp.RoutingModel(manager)

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        10000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    return get_routes(solution, routing, manager)

In [39]:
show_solution(3,Sample)

length of start list/ endlist does not match with number of vehicle
initializing start and end to 0
Start_list: [0, 0, 0]
Ends_list: [0, 0, 0]
Route for vehicle 0:
 0 ->  2 ->  12 ->  19 ->  31 ->  29 ->  10 ->  11 ->  15 ->  28 -> 0
Distance of the route: 52Km

Route for vehicle 1:
 0 ->  33 ->  27 ->  6 ->  20 ->  5 ->  18 ->  3 ->  9 ->  35 ->  16 ->  1 ->  36 ->  13 ->  39 ->  30 ->  32 ->  24 ->  7 -> 0
Distance of the route: 51Km

Route for vehicle 2:
 0 ->  38 ->  23 ->  8 ->  17 ->  26 ->  37 ->  4 ->  14 ->  25 ->  34 ->  22 ->  21 -> 0
Distance of the route: 48Km

Maximum of the route distances: 52Km


[[0, 2, 12, 19, 31, 29, 10, 11, 15, 28, 0],
 [0, 33, 27, 6, 20, 5, 18, 3, 9, 35, 16, 1, 36, 13, 39, 30, 32, 24, 7, 0],
 [0, 38, 23, 8, 17, 26, 37, 4, 14, 25, 34, 22, 21, 0]]

In [40]:
show_solution(3,Sample,[12,27,23],[0,0,0])

Start_list: [12, 27, 23]
Ends_list: [0, 0, 0]
Route for vehicle 0:
 12 ->  16 ->  11 ->  30 ->  32 ->  20 ->  13 ->  3 ->  33 ->  18 ->  24 ->  9 ->  36 ->  31 ->  17 ->  2 ->  37 ->  14 -> 0
Distance of the route: 32Km

Route for vehicle 1:
 27 ->  4 ->  19 ->  6 ->  10 ->  28 ->  38 ->  34 ->  25 ->  29 ->  22 -> 0
Distance of the route: 40Km

Route for vehicle 2:
 23 ->  35 ->  21 ->  7 ->  5 ->  15 ->  26 ->  8 ->  1 ->  39 -> 0
Distance of the route: 41Km

Maximum of the route distances: 41Km


[[12, 16, 11, 30, 32, 20, 13, 3, 33, 18, 24, 9, 36, 31, 17, 2, 37, 14, 0],
 [27, 4, 19, 6, 10, 28, 38, 34, 25, 29, 22, 0],
 [23, 35, 21, 7, 5, 15, 26, 8, 1, 39, 0]]

# Additional Features

### 1. Add/Remove a visit from a route
### 2.  Find Suitable Route for a visit

# 1. Add / Remove a visit form a Route.
    
    1. Add visit in a particular route:
        How?:
            find distance from other points in route and add it to distance matrix of a route.
            them find suitable path for a vehicle.
            
        Conditions:
            change starting point of Route from last completed request and end trip at Depot.
            
    1. Remove visit in a particular route:
        How?:
            Remove point from distance matrixs and then recalculate path.
            
        Conditions:
            change starting point of Route from last completed request and end trip at Depot.
        
       


## Modified VRP with start and end Point for Single Route

In [ ]:
 # Saving Routes
Original_Routes = routes[:]

In [ ]:
#routes = Original_Routes[:]

In [ ]:
def add_visit(route_number,visits_to_add = 1):
    df = pd.DataFrame()
    global Sample
    temp_data = []
    columns = list(Sample)
    index_temp = len(Sample)
    routes = Original_Routes
    if route_number != None or route_number <= routing.vehicles():
        
        print('Selected Route is:',route_number)
        df = Sample.iloc[routes[route_number]]
        for i in range(visits_to_add):
            placeholder = 1
            while placeholder is not None:
                try:
                    id_1 = int(input('Enter ID:'))
                    lat = float(input('Enter Latitude:'))
                    long = float(input('Enter Longitude:'))
                    pin = int(input('Enter Pincode:'))
                    placeholder = None
                except Exception as e:
                    placeholder = e
                    print('error occurred try again!')
                    continue
            
            values = [id_1, lat, long, pin]
            zipped = zip(columns, values)
            a_dictionary = dict(zipped)
            print(a_dictionary)
            temp_data.append(a_dictionary)
            #remove Comment below to append data to Sample Too
            Sample.loc[index_temp+1+i] = values
            df.loc[index_temp+1+i] = values
            #display(HTML(df.to_html()))
    else:
        print('error! check index of route')
    
    return df

In [ ]:
def update_routes(main_routes,updated_route,route_index):
    main_routes[route_index] = updated_route
    return main_routes

# Add Visit to a Route

#### Run Script Below to Add visit 

In [ ]:
print(routes)
route_number  = int(input(f'Enter Route index to Update:\n'))
no_of_visit  = int(input(f'Enter No of Visits to add in route:{route_number}\n'))

#Create Dataframe with exisiting route.
add_visit_df  = add_visit(route_number,visits_to_add = no_of_visit)
add_visit_df.drop_duplicates(inplace = True)
display(HTML(add_visit_df.to_html()))
print('Enter space seperated Index to mark as completed.')

# Inform Completed Visits
try:
    completed_visit = [int(x) for x in input().split()]
except:
    completed_visit = None
    print("Completed_visit is None")
    
#checking completed visit and removing depot from list
depot = 0
try :
    completed_visit.remove(depot)
except ValueError:
    print("{depot} Not Found in List")
    
# storing original index in 'index' column
add_visit_df['index'] = add_visit_df.index

# Removing Completed visits from DataFrame
add_visit_df = add_visit_df[~add_visit_df['index'].isin(completed_visit[:-1])]
add_visit_df = add_visit_df.reset_index()

# Initializing start index to reroute with completed visit list.
if completed_visit:
    start_index = add_visit_df[add_visit_df['index']==completed_visit[-1]].index.values[0]
else:
    start_index = 0

# Rerouting with Modified Dataframe
route_to_update  = show_solution(1,add_visit_df,start=[start_index])[0]

# Fixing indexs
index_dict = add_visit_df['index'].to_dict()

temp_route = []
for i in route_to_update:
    x = index_dict[i]
    temp_route.append(x)
route_to_update = temp_route[:]

print('route to update:',route_to_update)

#Routes of Main Solution
print('Original\n',Original_Routes[route_number])
#updated Solution
route_to_update = [0]+ completed_visit[:-1] + route_to_update
routes_x = update_routes(routes,route_to_update,route_number)
routes = routes_x[:]

# Removing Visit 

In [ ]:
def remove_visit(route_index):
    data = Sample.iloc[routes[route_index]]
    display(HTML(data.to_html()))
    print('Enter Index to Remove \n')
    list_to_remove = [int(x) for x in input().split()]
    print('Visits Removed:',list_to_remove)
    return data[~data.index.isin(list_to_remove)]

#### Run script below to remove visits

In [ ]:
print(routes)
route_number  = int(input(f'Enter Route index to Update:\n'))

#Create Dataframe with exisiting route and removing visits in it.
df  = remove_visit(route_number)
df.drop_duplicates(inplace = True)
display(HTML(df.to_html()))

# Inform Completed Visits
print('Enter space seperated Index to mark as completed.')
try:
    completed_visit = [int(x) for x in input().split()]
except:
    completed_visit = None
    print("Completed_visit is None")
# removing depot from list
depot = 0
try :
    completed_visit.remove(depot)
except ValueError:
    print("zero Not Found in List")
print('Completed Visits are', completed_visit)
# Storing Original Index in 'index' column 
df['index'] = df.index

# Removing Completed Visits from DataFrame
df = df[~df['index'].isin(completed_visit[:-1])]
df = df.reset_index()
if completed_visit:
    start_index = df[df['index']==completed_visit[-1]].index.values[0]
else:
    start_index = 0

# Rerouting with Modified Route Dataframe
route_to_update  = show_solution(1,df,start=[start_index])[0]

# Fxing Dataframe Index
index_dict = df['index'].to_dict()
temp_route = []
for i in route_to_update:
    x = index_dict[i]
    temp_route.append(x)
route_to_update = temp_route[:]

print('route to update:',route_to_update)

#Routes of Main Solution
print('Original\n',Original_Routes[route_number])
#updated Solution
route_to_update = [0]+ completed_visit[:-1] + route_to_update
routes_y = update_routes(routes,route_to_update,route_number)
# Updating Original Routes
routes = routes_y[:]

In [ ]:
Original_Routes[0]

In [ ]:
routes_y[0]


# Plotting Maps

In [ ]:
predefined_colors = ['#1890FF','#52c41a','#faad14','#ff4d4f','#69717a','#ff7f27']

## Adding Path to the Map

In [ ]:
import folium
import polyline

#### Extacting route path with start and end points. 

## Plotting Map with Path 

In [ ]:
def plot_routes(routes):
    gmap3 = gmplot.GoogleMapPlotter(23.1993477,77.2658056, 13)
    #ploting updated routes
    # routes = Original_Routes
    # scatter method of map object
    # scatter points on the google map
    for i in range(len(routes)):
        latitude_list = Sample.iloc[routes[i]]['Latitude'].values
        longitude_list = Sample.iloc[routes[i]]['Longitude'].values
        center_lat = latitude_list[0]
        center_long = longitude_list[0]
        #gmap3.scatter(center_lat , center_long, '# 00000',size = 100, marker = False )
        gmap3.scatter( latitude_list, longitude_list, '# FF0000',size = 40, marker = False )
        for j in range(len(latitude_list)):
            start_lat = latitude_list[j]
            start_long = longitude_list[j]
            #print(f'start_lat:{start_lat}, start_long{start_long}')
            try:
                stop_lat = latitude_list[j+1]
                stop_long = longitude_list[j+1]
                #print(f'stop_lat:{stop_lat}, stop_long{stop_long}')
            except:
                stop_lat = latitude_list[0]
                stop_long = longitude_list[0]
                #print('try triggered!')
                #print(f'stop_lat:{stop_lat}, stop_long{stop_long}')

            route_df = get_route(start_long, start_lat, stop_long, stop_lat)
            #print(len(route_df))
            route_latitude_list = route_df['Latitude'].values
            route_longitude_list = route_df['Longitude'].values
            #print(len(route_longitude_list))
    #         gmap3.scatter( route_latitude_list, route_longitude_list, '# FF0000',size = 0, marker = False )
            gmap3.plot(route_latitude_list, route_longitude_list,color=predefined_colors[i] , edge_width = 2.5)

    gmap3.draw( r"C:\Users\himan_ucrrloo\OneDrive\Desktop\x_routes.html" )
    print('Route Ploted on x_routes.html')

In [ ]:
#plot route plot
plot_routes(routes)

# Adding Time Window in VRP 

In [53]:
def open_tm(vehicle_no=4,data=None):
    Sample_x = data[:]
    str_x =''
    for x, y in zip(Sample_x['Latitude'], Sample_x['Longitude']):
        str_x+=f'{y},{x};'
    str_x = str_x[:-1]
    
    # url_y gives duration matrix
    url_y = f"http://router.project-osrm.org/table/v1/driving/{str_x}"      
    resp = requests.post(url_y)
    data_y = json.loads(resp.text)
    lol = data_y['durations']
    for i in range(len(lol)):
        for j in range(len(lol[i])):
            lol[i][j]=math.ceil(lol[i][j]/60)
    data = {}     
    data['time_matrix'] = lol[:]
    data['num_vehicles'] = vehicle_no
    data['depot'] = 0
    data['Id'] = Sample.to_dict()['Id']

    #adding Random Time Interval:

    data['time_windows'] = random_time_window(len(lol),0,540,30)
    return data

def random_time_window(number,start_time,end_time,time_interval):
    temp_list = [(0,0)]
    for x in range(number-1):
        i = random.randrange(start_time,end_time,time_interval)
        temp_list.append((i,i+time_interval))
    return temp_list[:]
        

In [54]:

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))




In [55]:

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = [
        [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
        [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
        [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
        [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
        [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
        [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
        [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
        [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
        [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
        [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
        [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
        [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
        [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
        [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
        [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
        [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
        [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]
    data['time_windows'] = [
        (0, 5),  # depot
        (7, 12),  # 1
        (10, 15),  # 2
        (16, 18),  # 3
        (10, 13),  # 4
        (0, 5),  # 5
        (5, 10),  # 6
        (0, 4),  # 7
        (5, 10),  # 8
        (0, 3),  # 9
        (10, 16),  # 10
        (10, 15),  # 11
        (0, 5),  # 12
        (5, 10),  # 13
        (7, 8),  # 14
        (10, 15),  # 15
        (11, 15),  # 16
    ]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data


In [56]:
# Create and register a transit callback.
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]


In [57]:
def get_cumul_data(solution, routing, dimension):
  """Get cumulative data from a dimension and store it in an array."""
  # Returns an array cumul_data whose i,j entry contains the minimum and
  # maximum of CumulVar for the dimension at the jth node on route :
  # - cumul_data[i][j][0] is the minimum.
  # - cumul_data[i][j][1] is the maximum.

    cumul_data = []
    for route_nbr in range(routing.vehicles()):
    route_data = []
    index = routing.Start(route_nbr)
    dim_var = dimension.CumulVar(index)
    route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        dim_var = dimension.CumulVar(index)
        route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    cumul_data.append(route_data)
    return cumul_data

IndentationError: unexpected indent (<ipython-input-57-68ae18003875>, line 8)

In [58]:
"""Solve the VRP with time windows."""
    
def show_solution_time(vehicle_no,df,start=[],end=[]):
    if len(start) !=vehicle_no or len(end) !=vehicle_no:
        print('Length of start and end list must be equal to number of vehicles ')
        print("Please Specify Start or End Points list or use \"0\" in place of list")
        
        return None
    
    data = open_tm(vehicle_no,df)
    #print(data)
    data['starts'] = start[:]
    data['ends']  = end[:]
    
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),data['num_vehicles'], data['starts'],data['ends'])
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        600,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    return get_cumul_data(solution, routing, dimension)

In [ ]:
show_solution_time(4,Sample,[0,0,0,0],[0,0,0,0])

{'time_matrix': [[0, 14, 11, 13, 13, 17, 12, 10, 10, 12, 19, 19, 10, 19, 14, 23, 11, 10, 18, 13, 12, 7, 5, 10, 3, 16, 9, 8, 9, 15, 12, 17, 10, 4, 19, 10, 15, 8, 2, 16], [13, 0, 5, 9, 14, 12, 12, 20, 7, 8, 11, 11, 5, 11, 15, 15, 4, 8, 13, 5, 12, 16, 14, 7, 15, 16, 9, 9, 18, 7, 5, 9, 6, 13, 19, 7, 3, 9, 14, 8], [10, 5, 0, 9, 12, 12, 11, 18, 6, 7, 12, 12, 3, 12, 14, 16, 3, 6, 13, 6, 12, 13, 11, 7, 12, 15, 8, 8, 15, 8, 4, 10, 3, 11, 19, 5, 7, 8, 11, 9], [14, 10, 9, 0, 8, 5, 5, 19, 6, 5, 19, 19, 11, 14, 9, 23, 8, 7, 6, 13, 5, 16, 15, 6, 14, 9, 8, 8, 19, 16, 8, 17, 7, 12, 12, 6, 12, 8, 14, 11], [13, 14, 13, 8, 0, 11, 5, 17, 9, 10, 23, 23, 14, 18, 3, 27, 12, 9, 12, 17, 6, 15, 14, 9, 13, 5, 7, 6, 18, 19, 12, 21, 10, 11, 13, 10, 16, 7, 14, 15], [17, 12, 13, 5, 10, 0, 7, 22, 9, 9, 22, 22, 14, 16, 11, 26, 12, 10, 2, 16, 7, 19, 18, 9, 17, 11, 11, 10, 22, 18, 12, 19, 11, 15, 11, 10, 14, 11, 18, 13], [11, 12, 11, 5, 5, 7, 0, 17, 7, 8, 21, 21, 12, 16, 6, 25, 10, 8, 8, 15, 2, 14, 12, 7, 11, 6, 5, 4, 1

In [ ]:
get_routes(solution, routing, manager)